In [1]:
import os.path as op
import numpy as np
import mne

In [2]:
subjects_dir = "../../data/datasets/eeg_fsaverage"
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation

In [3]:
import matplotlib
matplotlib.use( 'tkagg' )

plot_bem_kwargs = dict(
    subject=subject, subjects_dir=subjects_dir,
    brain_surfaces='white', orientation='coronal',
    slices=[50, 100, 150, 200])

mne.viz.plot_bem(**plot_bem_kwargs)

Using surface: /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/bem/inner_skull.surf
Using surface: /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/bem/outer_skull.surf
Using surface: /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/bem/outer_skin.surf


<MNEFigure size 780x260.5 with 4 Axes>

In [4]:
kind = "standard_1020"
montage = mne.channels.make_standard_montage(kind, head_size='auto')
info = mne.create_info(ch_names=montage.ch_names, sfreq=100., ch_types='eeg')
info.set_montage(montage)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, AF9, AF7, AF5, AF3, AF1, AFz, AF2, AF4, AF6, AF8, ...
 chs: 94 EEG
 custom_ref_applied: False
 dig: 97 items (3 Cardinal, 94 EEG)
 highpass: 0.0 Hz
 lowpass: 50.0 Hz
 meas_date: unspecified
 nchan: 94
 projs: []
 sfreq: 100.0 Hz
>

In [5]:
src = mne.setup_source_space(subject, spacing='oct4', add_dist='patch',
                             subjects_dir=subjects_dir)
print(src)

Setting up the source space with the following parameters:

SUBJECTS_DIR = /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage
Subject      = fsaverage
Surface      = white
Octahedron subdivision grade 4

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/surf/lh.white...
Mapping lh fsaverage -> oct (4) ...
    Triangle neighbors and vertex normals...
Loading geometry from /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/surf/lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 258/163842 selected to source space (oct = 4)

Loading /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/surf/rh.white...
Mapping rh fsaverage -> o

In [6]:
# mne.viz.plot_alignment(
#     info, src=src, eeg=['original', 'projected'], trans=trans,
#     show_axes=True, mri_fiducials=True, dig='fiducials')

In [7]:
# fig = mne.viz.plot_alignment(subject=subject, subjects_dir=subjects_dir,
#                              surfaces='white', coord_frame='mri',
#                              src=vol_src)
#
# mne.viz.set_3d_view(fig, azimuth=173.78, elevation=101.75,
#                     distance=0.30, focalpoint=(-0.03, -0.01, 0.03))

In [8]:
%%time
conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
outer skin  CM is  -0.21 -19.38  -0.23 mm
outer skull CM is  -0.19 -19.34  -0.49 mm
inner skull CM is  -0.53 -21.10   6.21 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    1.6 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is approximately    5.4 mm
Surfaces passed the basic topology checks.
Complete.

Approximation method : Linear collocation

Three-layer model surfaces loaded.
Computing the linear collocation s

In [10]:
surface = op.join(subjects_dir, subject, 'bem', 'inner_skull.surf')
vol_src = mne.setup_volume_source_space(
    subject, subjects_dir=subjects_dir,
    add_interpolator=False, bem = bem)  # Just for speed!
print(vol_src)

BEM              : <ConductorModel | BEM (3 layers)>
grid                  : 5.0 mm
mindist               : 5.0 mm
MRI volume            : /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/mri/T1.mgz

Reading /home/lauraflyra/Documents/BCCN/NI_project/neurolib/neurolib/models/eeg/../../data/datasets/eeg_fsaverage/fsaverage/mri/T1.mgz...

Taking inner skull from <ConductorModel | BEM (3 layers)>
Surface CM = (  -0.5  -21.1    6.2) mm
Surface fits inside a sphere with radius   98.3 mm
Surface extent:
    x =  -75.2 ...   76.3 mm
    y = -113.3 ...   75.0 mm
    z =  -71.7 ...   88.2 mm
Grid extent:
    x =  -80.0 ...   80.0 mm
    y = -115.0 ...   75.0 mm
    z =  -75.0 ...   90.0 mm
43758 sources before omitting any.
29824 sources after omitting infeasible sources not within 0.0 - 98.3 mm.
Source spaces are in MRI coordinates.
Checking that the sources are inside the surface and at least    5.0 mm away (will take a few...

In [11]:
fwd = mne.make_forward_solution(info, trans=trans, src=src, bem=bem,
                                meg=False, eeg=True, mindist=5.0, n_jobs=1,
                                verbose=True)
# raw.info
print(fwd)

Source space          : <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=258>, <surface (rh), n_vertices=163842, n_used=258>] MRI (surface RAS) coords, subject 'fsaverage', ~30.1 MB>
MRI -> head transform : /home/lauraflyra/anaconda3/envs/mne/lib/python3.10/site-packages/mne/data/fsaverage/fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 516 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  94 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Employing the head->MRI coordinate transform with the BEM model.
BEM model instan

In [12]:
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Leadfield size : 94 sensors x 1548 dipoles
